In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import devkit.api as dk
import pandas as pd
import numpy as np
from sklearn.decomposition import KernelPCA
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.tree import DecisionTreeRegressor  # 导入决策树分类器

In [ ]:
conn = dk.MySQLProxy()
conn.connect("root", "123888", "indicator")

In [ ]:
query = "SELECT * FROM report_variables1 WHERE (Year(date_report) BETWEEN 2011 AND 2013) AND MONTH(date_report) =12"
fr = conn.query_as_dataframe(query)

In [ ]:
fr['date_report'] = fr.date_report.apply(lambda x: str(x))
fr['date_published'] = fr.date_published.apply(lambda x: str(x))

In [ ]:
fr_final = fr.copy()

In [ ]:
fr_final.head(3)

In [ ]:
query = "SELECT date, sec_id, close, traded_days_until_now, is_trade, is_st, stock_float_shares \
FROM time_series_variables \
WHERE is_st=0 AND (date BETWEEN '2012-01-01' AND '2015-4-31') AND traded_days_until_now>400 AND is_trade=1 \
;"


In [ ]:
close1 = conn.query_as_dataframe(query)

In [ ]:
close1['date'] = close1.date.apply(lambda x: str(x))

In [ ]:
close1.head(3)

In [ ]:
close2 = close1[['date', 'sec_id', 'close', 'stock_float_shares']]

In [ ]:
temp1 = fr_final[['date_published','sec_id']].copy()

In [ ]:
temp1.head(3)

In [ ]:
def get_close_price_on_published_day(df_fr, df_close):
    """
    df_fr: <DataFrame>: columns = ['date_published','sec_id', 'date_benchmark']
    df_close: <DataFrame>: columns = [date sec_id  close stock_float_shares]
    type: <int>: close_beging 年报发布日当天或之后第一个交易日的股价 
                 close_ending 报发布日一年后最接近的股价
    """
    df_close['yearmonth'] = df_close.date.apply(lambda x: x[:7])
    df_fr['yearmonth'] = df_fr['date_published'].apply(lambda x: x[:7])
    df1=df_fr.merge(df_close, how='inner', on=['sec_id','yearmonth'])
    df2 = df1.groupby(['sec_id', 'date_published'], as_index=False).apply(lambda x: x[x.date >= x.date_published].head(1))
    df3 = df2.rename(columns={'close': 'close_begin','date':'date_beging'}).copy()
    del df3['yearmonth']
    return df3

In [ ]:
def get_close_price_after_one_year_on_published_day(df_fr, df_close):
    """
    df_fr: <DataFrame>: columns = ['date_published','sec_id']
    df_close: <DataFrame>: columns = [date sec_id  close stock_float_shares]

    """
    df_close['yearmonth'] = df_close.date.apply(lambda x: x[:7])
    df_fr['date_one_year'] = df_fr.date_published.apply(lambda x: str(int(x[:4]) + 1) + x[4:])  # 1年后的价格
    df_fr['yearmonth'] = df_fr['date_one_year'].apply(lambda x: x[:7])
    df1 = df_fr.merge(df_close, how='inner', on=['sec_id', 'yearmonth'])
    df2 = df1.groupby(['sec_id', 'date_published'], as_index=False).apply(lambda x: x[x.date <= x.date_one_year].tail(1))
    df3 = df2.rename(columns={'close': 'close_ending', 'date':'date_ending'})
    del df3['yearmonth']
    del df3['date_one_year']
    return df3

In [ ]:
aaa = get_close_price_on_published_day(df_fr=temp1.copy(), df_close=close2.copy())

In [ ]:
bbb = get_close_price_after_one_year_on_published_day(df_fr = aaa.copy(), df_close=close2.copy())

In [ ]:
ccc = bbb[['date_published', 'sec_id', 'date_beging', 'close_begin', 'date_ending', 'close_ending', 'stock_float_shares_x','stock_float_shares_y']].copy()

In [ ]:
ccc['return_rate']  = ccc['close_ending'] / ccc['close_begin'] - 1

In [ ]:
ccc["stock_float_shares_mean"] = (ccc["stock_float_shares_x"] + ccc["stock_float_shares_y"]) / 2

In [ ]:
df1 = fr_final.merge(ccc,how='inner',on=['date_published','sec_id'])

In [ ]:
df_final.to_csv(r"E:\99_daily\TODAY\df_final.csv",encoding='utf-8')

In [ ]:
df_final = pd.read_csv(r"E:\99_daily\TODAY\df_final.csv",index_col=0)

In [ ]:
industry = pd.read_csv(r"E:\99_daily\TODAY\industry_sw.csv")

In [ ]:
industry['sec_id'] = industry['sec_id'].apply(lambda x: x.split('.')[1]+x.split('.')[0] )

In [ ]:
industry_map = industry[['sec_id', 'industry_sw']]

In [ ]:
df_final.shape

In [ ]:
df2 = df1.merge(industry_map, how='inner', on=['sec_id'])

In [ ]:
df3 = df2[~df2.industry_sw.isin(['银行', '非银金融'])].copy()

In [ ]:
train_pre = df3.drop(['sec_id', 'date_report', 'date_published','date_beging','close_begin','date_ending','close_ending','stock_float_shares_x','stock_float_shares_y','stock_float_shares_mean','industry_sw'], axis=1)

In [ ]:
train_pre.shape

In [ ]:
retained_vars=(train_pre.isnull().sum()==0)
retained_vars=retained_vars[retained_vars].index.tolist()

In [ ]:
train_pre1 = train_pre.loc[:, retained_vars]

In [ ]:
train_pre1.to_csv(r"E:\99_daily\TODAY\train_pre1.csv")

In [ ]:
train_pre1=pd.read_csv(r"E:\99_daily\TODAY\train_pre1.csv", encoding='gbk')

In [ ]:
train_pre1.head(3)

In [ ]:
scale(train_pre1,copy=False)

In [ ]:
train_pre2 = train_pre1.copy()

In [ ]:
y = train_pre2.iloc[:, -1]
X = train_pre2.iloc[:, :-1]

In [ ]:
kpca=KernelPCA()
kpca.fit(X)

In [ ]:
get_we = lambda x: x / x.sum()
get_we(kpca.lambdas_)[:16].sum()

In [ ]:
X_transform = kpca.transform(X)[:, :16]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_transform, y, test_size=0.33)

In [ ]:
x_train

In [ ]:
svr_rbf = SVR(C=0.3)
svr_rbf.fit(diabetes_X_train, diabetes_y_train)

In [ ]:
print("训练集R2= ", svr_rbf.score(diabetes_X_train, diabetes_y_train))
print("测试集R2= ", svr_rbf.score(diabetes_X_test, diabetes_y_test))

In [ ]:
tree_regressor = DecisionTreeRegressor()
tree_regressor.fit(x_train, y_train)

In [ ]:
print("训练集R2= ", tree_regressor.score(x_train, y_train))
print("测试集R2= ", tree_regressor.score(x_test, y_test))

In [ ]:
from sklearn import linear_model

In [ ]:
clf = linear_model.Lasso(alpha=0.4)

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
print("训练集R2= ", clf.score(x_train, y_train))
print("测试集R2= ", clf.score(x_test, y_test))

In [ ]:
from sklearn import datasets, linear_model

In [ ]:
diabetes = datasets.load_diabetes()

In [ ]:
diabetes_X = diabetes.data[:, np.newaxis, 2]

In [ ]:
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

In [ ]:
diabetes_y_train = diabetes.target[:-20]
diabetes_y_test = diabetes.target[-20:]

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
regr.fit(x_train, y_train)

In [ ]:
print('Variance score: %.2f' % regr.score(x_test, y_test))

In [ ]:
y_test

In [ ]:
test_pred = svr_rbf.predict(x_test)

In [ ]:
svr_rbf.score(x_train, y_train)

In [ ]:
svr_rbf.score(x_test, y_test)

In [ ]:
y_predict=svr_rbf.predict(x_test_new)

In [ ]:
r2_score(y_train, train_pred)

In [ ]:
r2_score(y_test, test_pred)

In [ ]:
r2_score(y, y_predict)

In [ ]:
temp1['yearmonth'] = temp1.date_end.apply(lambda x: x[:7])

In [ ]:
temp1.head(3)

In [ ]:
close2=temp1.merge(close1, how='inner', on=['sec_id','yearmonth'])

In [ ]:
close3=close2.groupby(['sec_id','date_published'],as_index=False).apply(lambda x: x[x.date <= x.date_end].tail(1))

In [ ]:
close3 = close3.rename(columns={'close': 'close_end'})

In [ ]:
del close3['yearmonth']

In [ ]:
close3['yearmonth'] = close3.date_published.apply(lambda x: x[:7])

In [ ]:
del close3['date_end']

In [ ]:
close3 = close3.rename(columns={'date': 'date_end'})

In [ ]:
close4=close3.merge(close1, how='inner', on=['sec_id','yearmonth'])

In [ ]:
close5=close4.groupby(['sec_id','date_published'],as_index=False).apply(lambda x: x[x.date >= x.date_published].head(1))

In [ ]:
close6 = close5.rename(columns={'date': 'date_beging','close':'close_begin'})

In [ ]:
close7 = close6[['date_published', 'sec_id', 'date_beging', 'close_begin', 'date_end', 'close_end', 'stock_float_shares_x','stock_float_shares_y']]

In [ ]:
close7